<a href="https://colab.research.google.com/github/HwangHanJae/BigDataPratice/blob/main/2_week_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#작업형1

데이터 출처 :https://www.data.go.kr/data/15007122/fileData.do (참고, 데이터 수정)

데이터 설명 : 2018년도 성인의 건강검 진데이터 (흡연상태 1- 흡연, 0-비흡연 )

데이터 url : https://raw.githubusercontent.com/Datamanim/datarepo/main/smoke/train.csv

## Q1. 수축기혈압과 이완기 혈압기 수치의 차이를 새로운 컬럼("혈압차")으로 생성하고, 연령대 코드별 각 그룹 중 "혈압차"의 분산이 5번째로 큰 연령대 코드를 구하여라.

In [18]:
import pandas as pd
import numpy as np
import scipy as sp

df = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/smoke/train.csv")

df["혈압차"] = df["수축기혈압"] - df["이완기혈압"]

df.groupby("연령대코드(5세단위)")["혈압차"].var().sort_values(ascending=False)[4:5]


연령대코드(5세단위)
60    94.386922
Name: 혈압차, dtype: float64

## Q2. 비만도를 나타내는 지표인 WHtR는 허리둘레 / 키로 표현한다. 일반적으로 0.58이상이면 비만으로 분류한다. 데이터중 WHtR 지표상 비만인 인원의 남/여 비율을 구하여라

In [39]:
import pandas as pd
import numpy as np

df = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/smoke/train.csv")
df["WHtR"] = df["허리둘레"] / df["신장(5Cm단위)"]
df_WHrR = df.loc[df["WHtR"] >= 0.58]
m_ratio =  df_WHrR.loc[df_WHrR["성별코드"] == "M", "성별코드"].count() / df_WHrR["성별코드"].count()
f_ratio = df_WHrR.loc[df_WHrR["성별코드"] == "F", "성별코드"].count() / df_WHrR["성별코드"].count()

#data =  df_WHrR["성별코드"].value_counts()
#data["M"] / data["F]
m_ratio / f_ratio

1.1693877551020406

#작업형2

데이터 출처 :https://www.kaggle.com/anmolkumar/health-insurance-cross-sell-prediction

 (참고, 데이터 수정)

데이터 설명 : 자동차 보험 가입 예측

문제타입 : 분류유형

평가지표 : f1-score

trainData url : https://raw.githubusercontent.com/Datamanim/datarepo/main/insurance/train.csv

testData url : https://raw.githubusercontent.com/Datamanim/datarepo/main/insurance/test.csv

subData url : https://raw.githubusercontent.com/Datamanim/datarepo/main/insurance/submission.csv

In [55]:
trainData = "https://raw.githubusercontent.com/Datamanim/datarepo/main/insurance/train.csv"
testData = "https://raw.githubusercontent.com/Datamanim/datarepo/main/insurance/test.csv"
subData = "https://raw.githubusercontent.com/Datamanim/datarepo/main/insurance/submission.csv"

import pandas as pd
import numpy as np

train = pd.read_csv(trainData)
test = pd.read_csv(testData)
submission = pd.read_csv(subData)


In [56]:
#타겟데이터는 Response 분리
target = ["Response"]
X = train.drop(target, axis=1)
y = train[target]

X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 304887 entries, 0 to 304886
Data columns (total 11 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   id                    304887 non-null  int64  
 1   Gender                304887 non-null  object 
 2   Age                   304887 non-null  int64  
 3   Driving_License       304887 non-null  int64  
 4   Region_Code           304887 non-null  float64
 5   Previously_Insured    304887 non-null  int64  
 6   Vehicle_Age           304887 non-null  object 
 7   Vehicle_Damage        304887 non-null  object 
 8   Annual_Premium        304887 non-null  float64
 9   Policy_Sales_Channel  304887 non-null  float64
 10  Vintage               304887 non-null  int64  
dtypes: float64(3), int64(5), object(3)
memory usage: 25.6+ MB


In [57]:
#범주형 자료 : Gender, Vehicle_Age, Vehicle_Damage

print("Gender : ",X["Gender"].unique())
print()
print("Vehicle_Age : ", X["Vehicle_Age"].unique())
print()
print("Vehicle_Damage : ", X["Vehicle_Damage"].unique())

#범주형 자료를 one-hot-encoding으로 변환
columns = ["Gender", "Vehicle_Age", "Vehicle_Damage"]
X = pd.get_dummies(data=X, columns = columns)
test = pd.get_dummies(data=test, columns=columns)

X

Gender :  ['Female' 'Male']

Vehicle_Age :  ['< 1 Year' '1-2 Year' '> 2 Years']

Vehicle_Damage :  ['No' 'Yes']


,id,Age,Driving_License,Region_Code,Previously_Insured,Annual_Premium,Policy_Sales_Channel,Vintage,Gender_Female,Gender_Male,Vehicle_Age_1-2 Year,Vehicle_Age_< 1 Year,Vehicle_Age_> 2 Years,Vehicle_Damage_No,Vehicle_Damage_Yes
0,188957,25,1,17.0,1,38785.0,152.0,59,1,0,0,1,0,1,0
1,275631,20,1,22.0,0,2630.0,160.0,113,0,1,0,1,0,1,0
2,329036,40,1,41.0,0,33165.0,124.0,188,1,0,1,0,0,0,1
3,227288,28,1,30.0,1,29038.0,152.0,233,1,0,0,1,0,1,0
4,300441,50,1,28.0,0,31325.0,124.0,258,1,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
304882,351529,75,1,33.0,0,32412.0,26.0,296,0,1,1,0,0,0,1
304883,287917,57,1,28.0,1,44602.0,52.0,135,0,1,1,0,0,1,0
304884,21023,40,1,35.0,0,38779.0,26.0,293,1,0,1,0,0,0,1
304885,67340,24,1,41.0,0,26669.0,152.0,29,0,1,0,1,0,0,1


In [58]:
#필요없는 컬럼 제거 후에 스케일링(MinMaxScaler)

drop_col = ["id"]
X = X.drop(drop_col, axis=1)
test = test.drop(drop_col, axis=1)

from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

X = scaler.fit_transform(X)
test = scaler.fit_transform(test)

In [59]:
#훈련데이터, 테스트데이터 분리
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42, stratify=y)

#모델링
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier()

rf.fit(X_train,y_train)

y_pred = rf.predict(X_test)

#평가
from sklearn.metrics import f1_score

print(f1_score(y_test, y_pred))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  # This is added back by InteractiveShellApp.init_path()


0.1822818628939219


In [63]:
#제출
pred = rf.predict(test)

submission["0"] = pred

submission.to_csv("수험번호.csv", index=False)